In [1]:
def read_csv(filename, hasHeader=False):
    data = []
    import csv
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        if (hasHeader):
            next(reader, None)
            
        for row in reader:
            data.append(row)
    
    return data

In [17]:
items = read_csv('data/ItemInfo_train.csv', hasHeader=True)
print 'Train items', len(items)
print 'Item example', items[0]

Train items 3344613
Item example ['1', '81', '\xd0\x9f\xd1\x80\xd0\xbe\xd0\xb4\xd0\xb0\xd0\xbc \xd0\x9a\xd0\xb0\xd0\xbc\xd0\xb0\xd0\xb7 6520', '\xd0\x9f\xd1\x80\xd0\xbe\xd0\xb4\xd0\xb0\xd0\xbc \xd0\x9a\xd0\xb0\xd0\xbc\xd0\xb0\xd0\xb7 6520 20 \xd1\x82\xd0\xbe\xd0\xbd\xd0\xbd', '1064094, 5252822, 6645873, 6960145, 9230265', '{"\xd0\x92\xd0\xb8\xd0\xb4 \xd1\x82\xd0\xb5\xd1\x85\xd0\xbd\xd0\xb8\xd0\xba\xd0\xb8":"\xd0\x93\xd1\x80\xd1\x83\xd0\xb7\xd0\xbe\xd0\xb2\xd0\xb8\xd0\xba\xd0\xb8"}', '300000.0', '648140', '', '64.686946', '30.815924']


In [18]:
COL_ITEM_ID=0
COL_CATEGORY_ID=1
COL_TITLE=2
COL_DESCRIPTION=3
COL_IMAGES=4
COL_JSON=5
COL_PRICE=6
COL_LOCATION=7
COL_METRO=8
COL_LAT=9
COL_LON=10

In [19]:
map_items = dict()
for item in items:
    map_items[item[COL_ITEM_ID]] = item

In [20]:
pairs = read_csv('data/ItemPairs_train.csv', hasHeader=True)
print 'Train pairs', len(pairs)
print 'Example pair', pairs[0]

Train pairs 2991396
Example pair ['1', '4112648', '1', '1']


In [21]:
categories = read_csv('data/Category.csv')
print 'Categories', len(categories)

Categories 52


In [22]:
map_parent_category = dict()
for category in categories:
    map_parent_category[category[0]] = category[1]

In [23]:
locations = read_csv('data/Location.csv', hasHeader=True)
print 'Locations', len(locations)

Locations 3449


In [24]:
map_regions = dict()
for location in locations:
    map_regions[location[0]] = location[1]

In [25]:
def same_category(item1, item2):
    cat1 = item1[COL_CATEGORY_ID]
    cat2 = item2[COL_CATEGORY_ID]
    
    return cat1 == cat2

def same_parent_category(item1, item2):
    cat1 = item1[COL_CATEGORY_ID]
    cat2 = item2[COL_CATEGORY_ID]
    
    par_cat1 = map_parent_category[cat1]
    par_cat2 = map_parent_category[cat2]
    
    return par_cat1 == par_cat2

def same_price(item1, item2):
    price1 = item1[COL_PRICE]
    price2 = item2[COL_PRICE]
    
    return price1 == price2

def same_lat(item1, item2):
    lat1 = item1[COL_LAT]
    lat2 = item2[COL_LAT]
    
    return lat1 == lat2

def same_lon(item1, item2):
    lon1 = item1[COL_LON]
    lon2 = item2[COL_LON]
    
    return lon1 == lon2

def same_location(item1, item2):
    location1 = item1[COL_LOCATION]
    location2 = item2[COL_LOCATION]
    
    return location1 == location2

def same_region(item1, item2):
    location1 = item1[COL_LOCATION]
    location2 = item2[COL_LOCATION]
    
    region1 = map_regions[location1]
    region2 = map_regions[location2]
    
    return region1 == region2

def same_metro(item1, item2):
    metro1 = item1[COL_METRO]
    metro2 = item2[COL_METRO]
    
    return metro1 == metro2

def same_words(str1, str2):
    return False

def get_features(item1, item2, label):
    fx = []
    fx.append(same_category(item1, item2))
    fx.append(same_parent_category(item1, item2))
    fx.append(same_price(item1, item2))
    fx.append(same_lat(item1, item2))
    fx.append(same_lon(item1, item2))
    fx.append(same_location(item1, item2))
    fx.append(same_region(item1, item2))
    fx.append(same_metro(item1, item2))
    
    return (fx, label)

In [29]:
def get_train_data():
    data = []
    for pair in pairs:
        item1 = map_items[pair[0]]
        item2 = map_items[pair[1]]
        label = int(pair[2])

        xy = get_features(item1, item2, label)
        data.append(xy)
        
    return data

train = get_train_data()
print train[0]

([True, True, True, True, True, True, True, True], 1)


In [30]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(train, test_size=0.20, train_size=0.80)

train_x = [x[0] for x in train_data]
train_y = [x[1] for x in train_data]

test_x = [x[0] for x in test_data]
test_y = [x[1] for x in test_data]

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(train_x, train_y)

print classifier.score(test_x, test_y)

0.671837601123
